In [1]:
!pip install -q streamlit pyngrok scikit-learn seaborn joblib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from pyngrok import ngrok

ngrok.set_auth_token("36nIAv5i870cFQbZXOps4LDxs91_61KKwf9Cypzx2R5bpnGJq")

In [6]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/heart_disease_uci.csv")

print(f"Sample dataset created with {len(df)} records")
print(f"Disease distribution: {df['num'].value_counts().to_dict()}")

Sample dataset created with 920 records
Disease distribution: {0: 411, 1: 265, 2: 109, 3: 107, 4: 28}


In [10]:
#Create the app.py file
app_code = '''import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
from io import BytesIO

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

#loading data
def load_data(uploaded_file=None, path=None):
    if uploaded_file is not None:
        return pd.read_csv(uploaded_file)
    elif path and os.path.exists(path):
        return pd.read_csv(path)
    return None

#preprocessing
def preprocess(df, target_col="num"):
    df = df.copy()
    df = df.drop_duplicates().reset_index(drop=True)
    num_cols = df.select_dtypes(include=np.number).columns
    for c in num_cols:
        df[c] = df[c].fillna(df[c].median())
    if target_col in df.columns:
        if df[target_col].nunique() > 2:
            df[target_col] = df[target_col].apply(lambda x: 1 if x > 0 else 0)
    return df

#getting features and target
def get_features_targets(df, target_col="num"):
    X = df.drop(columns=[target_col])
    y = df[target_col]
    return X, y

#one-hot encoding
def encode_categorical(X_train, X_test):
    X_train_enc = pd.get_dummies(X_train)
    X_test_enc = pd.get_dummies(X_test)
    X_test_enc = X_test_enc.reindex(columns=X_train_enc.columns, fill_value=0)
    return X_train_enc, X_test_enc

#feature scaling
def scale_data(X_train, X_test):
    scaler = StandardScaler()
    return scaler.fit_transform(X_train), scaler.transform(X_test), scaler

#model training
def train_model(model_name, X_train, y_train, **kwargs):
    if model_name == "Logistic Regression":
        model = LogisticRegression(max_iter=1000, **kwargs)
    elif model_name == "KNN":
        model = KNeighborsClassifier(**kwargs)
    elif model_name == "Random Forest":
        model = RandomForestClassifier(**kwargs)
    elif model_name == "SVM":
        model = SVC(probability=True, **kwargs)
    model.fit(X_train, y_train)
    return model

#evaluation
def plot_confusion(cm, labels):
    fig, ax = plt.subplots(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, ax=ax)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("Actual")
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)
    return fig

#saving model
def save_model_to_session(model, scaler, feature_columns):
    st.session_state["trained_model"] = model
    st.session_state["scaler"] = scaler
    st.session_state["feature_columns"] = feature_columns
    st.session_state["model_trained"] = True

#download option
def download_model():
    model_data = {
        "model": st.session_state.get("trained_model"),
        "scaler": st.session_state.get("scaler"),
        "feature_columns": st.session_state.get("feature_columns")
    }
    buffer = BytesIO()
    joblib.dump(model_data, buffer)
    buffer.seek(0)
    return buffer

#app layout and stuff
st.set_page_config(page_title="Heart Disease Predictor", layout="wide", page_icon="🫀")

st.markdown("<h1 style=\\"color: #e74c3c; text-align: center;\\">🫀 Heart Disease Prediction System</h1>", unsafe_allow_html=True)
st.markdown("### Python Project - ML-Based Diagnostic Support")

if "model_trained" not in st.session_state:
    st.session_state["model_trained"] = False

st.sidebar.title("⏣ Configuration")
st.sidebar.header("1️. Load Dataset")
uploaded_file = st.sidebar.file_uploader("Upload CSV dataset", type=["csv"])

df = None
if uploaded_file:
    df = load_data(uploaded_file=uploaded_file)
elif os.path.exists("heart_disease_uci.csv"):
    df = load_data(path="heart_disease_uci.csv")
    st.sidebar.info("Using local dataset")

if df is None:
    st.warning("▷ Please upload a dataset to get started!")
    st.stop()

target_col = st.sidebar.text_input("Target column:", value="num")

try:
    df = preprocess(df, target_col=target_col)
    st.sidebar.success(f"{df.shape[0]} rows × {df.shape[1]} cols")
except Exception as e:
    st.sidebar.error(f"Error: {e}")
    st.stop()

tab1, tab2, tab3, tab4 = st.tabs(["🗐 Data", "⌯⌲ Training", "𖡊 Predict", "ⓘ About"])

with tab1:
    st.header("Dataset Overview")
    col1, col2, col3 = st.columns(3)
    col1.metric("Samples", df.shape[0])
    col2.metric("Features", df.shape[1] - 1)
    col3.metric("Classes", df[target_col].nunique())
    st.dataframe(df.head(10), use_container_width=True)

    col_left, col_right = st.columns(2)
    with col_left:
        st.subheader("Summary Stats")
        st.dataframe(df.describe())
    with col_right:
        st.subheader("Target Distribution")
        target_counts = df[target_col].value_counts()
        st.write(target_counts)
        fig, ax = plt.subplots(figsize=(6, 6))
        target_counts.plot(kind="pie", autopct="%1.1f%%", labels=["No Disease", "Disease"], colors=["#2ecc71", "#e74c3c"], explode=[0.05, 0], ax=ax)
        ax.set_ylabel("")
        st.pyplot(fig)

    if st.checkbox("Show Correlation Heatmap"):
        numeric_df = df.select_dtypes(include=[np.number])
        fig, ax = plt.subplots(figsize=(12, 10))
        sns.heatmap(numeric_df.corr(), annot=True, fmt=".2f", cmap="RdBu_r", center=0, ax=ax)
        st.pyplot(fig)

with tab2:
    st.header("Train ML Model")
    X, y = get_features_targets(df, target_col=target_col)
    all_features = X.columns.tolist()
    selected_features = st.multiselect("Select features:", all_features, default=all_features)

    if len(selected_features) == 0:
        st.warning("Select at least one feature!")
        st.stop()

    X_sel = X[selected_features]

    col1, col2 = st.columns(2)
    with col1:
        test_size = st.slider("Test size (%):", 10, 50, 20) / 100.0
    with col2:
        random_state = st.number_input("Random seed:", value=42)

    X_train, X_test, y_train, y_test = train_test_split(X_sel, y, test_size=test_size, random_state=int(random_state), stratify=y)
    st.info(f"Training: {len(X_train)} | Test: {len(X_test)}")

    X_train_enc, X_test_enc = encode_categorical(X_train, X_test)
    X_train_scaled, X_test_scaled, scaler = scale_data(X_train_enc, X_test_enc)

    model_name = st.selectbox("Algorithm:", ["Logistic Regression", "Random Forest", "KNN", "SVM"])

    kwargs = {}
    if model_name == "Logistic Regression":
        c = st.slider("C:", 0.01, 10.0, 1.0)
        kwargs = {"C": c, "solver": "lbfgs"}
    elif model_name == "Random Forest":
        n_est = st.slider("Trees:", 10, 200, 100)
        max_d = st.slider("Max depth (0=None):", 0, 30, 10)
        kwargs = {"n_estimators": int(n_est), "random_state": int(random_state)}
        if max_d > 0:
            kwargs["max_depth"] = int(max_d)
    elif model_name == "KNN":
        n_neigh = st.slider("Neighbors:", 1, 25, 5)
        kwargs = {"n_neighbors": int(n_neigh)}
    elif model_name == "SVM":
        c = st.slider("C:", 0.01, 10.0, 1.0)
        kernel = st.selectbox("Kernel:", ["rbf", "linear", "poly"])
        kwargs = {"C": c, "kernel": kernel}

    if st.button("⌯⌲ Train", type="primary", use_container_width=True):
        with st.spinner("Training..."):
            try:
                model = train_model(model_name, X_train_scaled, y_train, **kwargs)
                save_model_to_session(model, scaler, X_train_enc.columns)
                y_pred = model.predict(X_test_scaled)
                acc = accuracy_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)
                st.success("Model trained!")
                col1, col2 = st.columns(2)
                col1.metric("Accuracy", f"{acc:.2%}")
                col2.metric("F1-Score", f"{f1:.4f}")
                st.text(classification_report(y_test, y_pred, target_names=["No Disease", "Disease"]))
                cm = confusion_matrix(y_test, y_pred)
                st.pyplot(plot_confusion(cm, ["No Disease", "Disease"]))
                st.download_button("🡻 Download", download_model(), "model.joblib")
            except Exception as e:
                st.error(f"Error: {e}")

with tab3:
    st.header("Make Predictions")
    if not st.session_state.get("model_trained"):
        st.warning("Train a model first!")
        st.stop()

    model = st.session_state["trained_model"]
    scaler = st.session_state["scaler"]
    feature_columns = st.session_state["feature_columns"]

    input_data = {}
    cols = st.columns(3)

    for idx, feat in enumerate(selected_features):
        col = cols[idx % 3]
        sample = X_sel[feat]
        if sample.dtype == "object":
            options = sorted(sample.dropna().unique().tolist())
            input_data[feat] = col.selectbox(f"**{feat}**", options, key=f"p_{feat}")
        else:
            input_data[feat] = col.number_input(f"**{feat}**", value=float(sample.mean()), key=f"p_{feat}")

    if st.button("🔎︎ Predict", type="primary"):
        try:
            inp_df = pd.DataFrame([input_data])[selected_features]
            inp_df_enc = pd.get_dummies(inp_df)
            inp_df_enc = inp_df_enc.reindex(columns=feature_columns, fill_value=0)
            inp_scaled = scaler.transform(inp_df_enc)
            pred = model.predict(inp_scaled)[0]
            pred_proba = model.predict_proba(inp_scaled)[0] if hasattr(model, "predict_proba") else None

            if pred == 1:
                st.error("### ⚠ Heart Disease Detected")
            else:
                st.success("### ⛉ No Heart Disease")

            if pred_proba is not None:
                col1, col2 = st.columns(2)
                col1.metric("No Disease", f"{pred_proba[0]:.1%}")
                col2.metric("Disease", f"{pred_proba[1]:.1%}")
        except Exception as e:
            st.error(f"Error: {e}")

with tab4:
    st.header("About")
    st.markdown("""
    ### Heart Disease Prediction System

    **Features:**
    - Multiple ML algorithms
    - Interactive visualizations
    - Real-time predictions
    - Model download

    **Tech Stack:**
    Streamlit, Scikit-learn, Pandas, NumPy
    """)

st.markdown("---")
st.markdown("<p style='text-align: center;'>Heart Disease Prediction | Python Project</p>", unsafe_allow_html=True)
'''

# Write the file
with open('app.py', 'w') as f:
    f.write(app_code)

print("app.py file created!")

app.py file created successfully!


In [11]:
from pyngrok import ngrok
import time

# Clean up
!pkill -f streamlit
time.sleep(2)

# Start tunnel
public_url = ngrok.connect(8501)

print("\n" + "="*70)
print("APP IS LIVE!")
print("="*70)
print(f"\nURL: {public_url}")
print("\nKeep this cell running!")
print("="*70 + "\n")

# Launch app
!streamlit run app.py --server.port 8501 --server.headless true


APP IS LIVE!

URL: NgrokTunnel: "https://unbeneficed-bicompact-eleni.ngrok-free.dev" -> "http://localhost:8501"

Keep this cell running!




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.145.223.23:8501

2025-12-13 17:01:09.685 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2025-12-13 17:01:27.715 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2025-12-13 17:01:37.057 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_